### Clustering the neighborhoods

This notebook follows the following steps:
1. Import the required packages
2. Loading the Data
3. Creating Maps
4. Foursquare API Usage
5. Data Extraction and Data Cleaning
6. K Means Clustering

### Step 1: Import the required packages

In [1]:
# ----- Basic utilities -----
import re
import json
import requests
import urllib

# ----- Data Manipulation Packages -----
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# ----- Visualization Packages -----
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# ----- Geo Coordinates Packages -----
from geopy.geocoders import Nominatim
import geocoder

# ----- Machine Learning Packages -----
from sklearn.cluster import KMeans

%matplotlib inline

### Step 2: Loading the Data

Load the [Geo Dataset](https://github.com/KrishnaChaitanya1/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering/Data%20Files/Geodata.csv) from the previous notebook

In [2]:
df = pd.read_csv('Geodata.csv')
df.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [3]:
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [4]:
df = df.sort_values(by = ['PostalCode'], ascending = True).reset_index(drop = True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Step 3: Creating Maps

Get the Latitude and Longitude of Toronto, Canada

In [5]:
address = 'Toronto, ON'

# user_agent attribute is used just to remove the warning generated otherwise
geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Latitude: {}, Longitude: {}".format(latitude, longitude))

Latitude: 43.653963, Longitude: -79.387207


Using folium, generate the map of Toronto using the latitude and longitude.

In [6]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng], 
    radius = 5, 
    popup = label, 
    color = 'blue', 
    fill = True, 
    fill_color = '#3186cc', 
    fill_opacity = 0.7).add_to(map_toronto)
    
map_toronto

For this analysis, I'm considering the Scarborough Borough, so isolate all the Scarborough values from the original dataframe.

In [7]:
scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop = True)
scarborough_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [8]:
address = 'Scarborough, ON'
geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Latitude: {}, Longitude: {}'.format(latitude, longitude))

Latitude: 43.773077, Longitude: -79.257774


Using folium generate the map of Scarborough Borough.

In [9]:
map_scar = folium.Map(location = [latitude, longitude], zoom_start = 10)
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng], 
    radius = 5, 
    popup = label, 
    color = 'blue', 
    fill = True, 
    fill_color = '#3186cc', 
    fill_opacity = 0.7).add_to(map_scar)
map_scar

### Step 4: Foursquare API Usage

Input the CLIENT_ID, CLIENT_SECRET and VERSION in order to start using the Foursquare API.

In [10]:
CLIENT_ID = 'BL4OMDGY22KNROQIDK1SH2B0HKWTRZRDTXUTZVQBOJ1YZXLP' # Change when replicating
CLIENT_SECRET = 'WUYFQMZ2L1CN512RXLUIZCWDBDPNBLR5EAMAWDYDJIZ3QEEY' # Change when replicating
VERSION = '20181204'

print("My Credentials: ")
print("CLIENT_ID: " + CLIENT_ID)
print("CLIENT_SECRET: " + CLIENT_SECRET)

My Credentials: 
CLIENT_ID: BL4OMDGY22KNROQIDK1SH2B0HKWTRZRDTXUTZVQBOJ1YZXLP
CLIENT_SECRET: WUYFQMZ2L1CN512RXLUIZCWDBDPNBLR5EAMAWDYDJIZ3QEEY


In [11]:
scarborough_data.loc[0, 'Neighborhood'] # get the first entry in the neighborhood column

'Rouge, Malvern'

In [12]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # get latitude of first entry
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] #get longitude of first entry
neighborhood_name = scarborough_data.loc[0, 'Neighborhood']

print("Latitude: {}, \nLongitude: {}, \nNeighborhood Name: {}".format(neighborhood_latitude, 
                       neighborhood_longitude, 
                       neighborhood_name))

Latitude: 43.8066863, 
Longitude: -79.19435340000003, 
Neighborhood Name: Rouge, Malvern


In [13]:
LIMIT = 100
# Radius is made 1000 because, with radius as 500, the results generated are not much.
radius = 1000

# using the url convention, create the url to get the requests
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=BL4OMDGY22KNROQIDK1SH2B0HKWTRZRDTXUTZVQBOJ1YZXLP&client_secret=WUYFQMZ2L1CN512RXLUIZCWDBDPNBLR5EAMAWDYDJIZ3QEEY&v=20181204&ll=43.8066863,-79.19435340000003&radius=1000&limit=100'

In [14]:
results = requests.get(url).json() # Get the data in JSON format
results

{'meta': {'code': 200, 'requestId': '5c0763c24c1f67195e6553d1'},
 'response': {'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 15,
  'suggestedBounds': {'ne': {'lat': 43.815686309000014,
    'lng': -79.18190576146083},
   'sw': {'lat': 43.79768629099999, 'lng': -79.20680103853923}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80228301948931,
          'lng': -79.19856472801668}],
       

### Step 5: Data Extraction and Data Cleaning

Function to isolate the categories.

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Convert the json to a dataframe by normalizing and extract the nearby columns.

In [16]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # Flatten the JSON to tabular data

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
2,Wendy's,Fast Food Restaurant,43.802008,-79.198080
3,Harvey's,Fast Food Restaurant,43.800106,-79.198258
4,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [17]:
print("Venues nearby: {}".format(nearby_venues.shape[0]))

Venues nearby: 15


Function to get the nearby venues of all the neighborhoods in Scarborough Borough area.

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


In [20]:
print(scarborough_venues.shape)
scarborough_venues.head()

(84, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [21]:
scarborough_venues.loc[-1] = ['Upper Rouge', 43.83455, -79.2069, "Al's Wine", 43.83455, -79.2069, "Bar"]

In [22]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",8,8,8,8,8,8
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"East Birchmount Park, Ionview, Kennedy Park",7,7,7,7,7,7


In [23]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 54 uniques categories.


Create dummies for various types of venues.

In [24]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix = '', prefix_sep = "")

scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood']

fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Golf Course,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant,Women's Store
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Golf Course,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant,Women's Store
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.0,0.0,0.00,0.000000,0.000000,0.25,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.0,0.000000,0.000000,0.000000,0.0
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.333333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000,0.333333,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.25,0.000000,0.000000,0.00,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.0,0.000000,0.000000,0.000000,0.0
3,Cedarbrae,0.0,0.142857,0.142857,0.142857,0.0,0.000000,0.0,0.0,0.00,0.142857,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.0,0.000000,0.142857,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.142857,0.000000,0.000000,0.0
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.1,0.000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.1,0.000000,0.00,0.0,0.000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.1,0.000000,0.000000,0.000000,0.0
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.125000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.125,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.125,0.000000,0.000000,0.125000,0.250000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.125000,0.000000,0.000000,0.0
6,"Cliffcrest, Cliffside, Scarborough Village West",0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.5,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.142857,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000

Get the top 5 venues in the neighborhoods and their frequency.

In [26]:
num_top_venues = 5

for hood  in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0       Breakfast Spot  0.25
1               Lounge  0.25
2         Skating Rink  0.25
3       Clothing Store  0.25
4  American Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0                 Park  0.33
1           Playground  0.33
2          Coffee Shop  0.33
3  American Restaurant  0.00
4    Korean Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1                   Café  0.25
2  General Entertainment  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                 venue  freq
0               Bakery  0.14
1                 Bank  0.14
2   Athletics & Sports  0.14
3      Thai Restaurant  0.14
4  Fried Chicken Joint  0.14


----Clairlea, Golden Mile, Oakridge----
          venue  freq
0        Bakery   0.2
1      Bus Line   0.2
2  Intersection   0.1
3          Park   0

Function to return most common venues

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Women's Store,Convenience Store,Grocery Store,Golf Course,General Entertainment,Fried Chicken Joint
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Coffee Shop,Playground,Park,Women's Store,Grocery Store,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Women's Store,Grocery Store,Golf Course,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store,Golf Course
4,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Bus Station,Park,Fast Food Restaurant,Metro Station,Intersection,Soccer Field,General Entertainment,Fried Chicken Joint
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Noodle House,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Pharmacy,Chinese Restaurant,Coffee Shop,General Entertainment,Fast Food Restaurant
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,College Stadium,Grocery Store,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Light Rail Station,Chinese Restaurant,Pet Store,Convenience Store,Golf Course,General Entertainment,Fried Chicken Joint
8,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Train Station,Department Store,Convenience Store,Hobby Shop,Coffee Shop,Women's Store,College Stadium,Grocery Store,Golf Course
9,"Guildwood, Morningside, West Hill",Breakfast Spot,Electronics Store,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Women's Store,College Stadium,Golf Course,General Entertainment


### Step 6: K Means Clustering

Onto Clustering. Start with 5 clusters.

In [29]:
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 0, 1, 1, 4, 2, 0, 2])

Add the cluster labels to the scarborough data

In [30]:
scarborough_merged = scarborough_data

scarborough_merged['Cluster Labels'] = kmeans.labels_

scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Women's Store,College Stadium,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Golf Course,Bar,Women's Store,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Breakfast Spot,Electronics Store,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Women's Store,College Stadium,Golf Course,General Entertainment
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Pharmacy,College Stadium,Grocery Store,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store,Golf Course


Finally plot the clusters.

In [31]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters